### Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

* <a href="#item1">Scrape and Explore wiki Toronto Table  </a>

* <a href="#item2">Choice of Borough in Toronto, North York  </a>

* <a href="#item3">Explore Neighborhoods in North York, Canada  </a>

* <a href="#item3">Analyze each Neighborhoods in North York, Canada  </a>

* <a href="#item4">Cluster Neighborhoods  </a>

* <a href="#item5">Examine Clusters  </a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import geocoder # import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# imprt folium
import folium # map rendering library

# import BeautifulSoup for data scraping
from bs4 import BeautifulSoup

# import csv to make csv files from scrapped data
import csv

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

** Import the Data **

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":862527922,"wgRevisionId":862527922,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [3]:
title = soup.title.text
print(title)

List of postal codes of Canada: M - Wikipedia


** Convert the table to a pandas Dataframe**

In [4]:
import pandas as pd
import numpy as np

dataSheet= []

for rows in soup.find_all('tr'):
    cell_content = rows.find_all('td')
    
    
    if len(cell_content) == 3:
        Postcode = cell_content[0].text.strip()
        Borough = cell_content[1].text.strip()
        Neighbourhood = cell_content[2].text.strip()
        
        
        if Borough == 'Not assigned':
            next
        else:  
            data = [Postcode, Borough, Neighbourhood]
            dataSheet.append(data)
df1 = pd.DataFrame(dataSheet, columns=['Postcode','Borough', 'Neighbourhood'])
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Replacing in neighborhoods that  are "Not assigned" with Name of the Borough  **

In [5]:
df1["Neighbourhood"] = df1["Neighbourhood"].replace("Not assigned", df1["Borough"])

** Combine the multiple neighborhoods**

In [6]:
df2 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

**Dataframe shape**

In [7]:
df2.shape

(103, 3)

** getting the corrdiniates of postal codes  using geocoder **

In [8]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


ModuleNotFoundError: No module named 'geocoder'

Since geocoder is not working Geospatial_data downloaded from the link

In [10]:

coord = pd.read_csv('http://cocl.us/Geospatial_data')

# merge df2 and coord dataframes

toronto_df = df2.merge(coord.rename(columns={'Postal Code':'Postcode'}),how='left', on = ['Postcode'])
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [32]:
toronto_df.shape

(103, 5)

## Choice of Borough in Toronto

#### Create a map of Toronto with neighborhoods superimposed on top.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


**Use geopy library to get the latitude and longitude values of Toronto.**

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
# north_york_df = df4[df4['Borough'].str.contains('North York')].reset_index(drop = True)

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
map_toronto

** Let's get the geographical coordinates of North York.**

In [14]:
north_york_df = toronto_df[toronto_df['Borough'].str.contains('North York')].reset_index(drop = True)

neighborhood_latitude = north_york_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


Let's get the geographical coordinates of Manhattan.

In [15]:
address = 'North York, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York, Canada are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of North York, Canada are 43.7709163, -79.4124102.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

In [16]:
# create map of Manhattan using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(north_york_df['Latitude'], north_york_df['Longitude'], north_york_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)  
    
map_northyork

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [1]:
CLIENT_ID = 'XXXXXXXXXXXXXXXXX' # your Foursquare ID
CLIENT_SECRET = 'XXXXXXXXXX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XXXXXXXXXXXXXXXXX
CLIENT_SECRET:XXXXXXXXXX


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [60]:
north_york_df.loc[0, 'Neighbourhood']

'Hillcrest Village'

Get the neighborhood's latitude and longitude values.

In [19]:
neighborhood_latitude = north_york_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = north_york_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Hillcrest Village are 43.8037622, -79.3634517.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [2]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    '20180605', 
    43.8037622, -79.3634517, 
    radius, 
    LIMIT)

Send the GET request and examine the resutls

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bd3e83b4434b912a28f1dd0'},
 'response': {'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.808262204500004,
    'lng': -79.3572281853783},
   'sw': {'lat': 43.7992621955, 'lng': -79.3696752146217}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad9dce6f964a520651b21e3',
       'name': "Eagle's Nest Golf Club",
       'location': {'address': '10000 Dufferin Rd',
        'lat': 43.805454826002794,
        'lng': -79.36418592243415,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.805454826002794,
          'lng': -79.36418592243415}],
        'distance': 197,
        'cc': 'CA',
        'city': 'Toronto

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [22]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Eagle's Nest Golf Club,Golf Course,43.805455,-79.364186
1,AY Jackson Pool,Pool,43.804515,-79.366138
2,Villa Madina,Mediterranean Restaurant,43.801685,-79.363938
3,Duncan Creek Park,Dog Run,43.805539,-79.360695
4,A.Y. Jackson Secondary School Track,Athletics & Sports,43.805068,-79.366677


And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


<a id='item2'></a>

## 2. Explore Neighborhoods in North York, Canada

#### Let's create a function to repeat the same process to all the neighborhoods in North York, Canada

In [61]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *manhattan_venues*.

In [40]:
# type your answer here

north_york_venues = getNearbyVenues(names=north_york_df['Neighbourhood'],
                                   latitudes=north_york_df['Latitude'],
                                   longitudes=north_york_df['Longitude'])


Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Maple Leaf Park, North Park, Upwood Park
Humber Summit
Emery, Humberlea


#### Let's check the size of the resulting dataframe

In [41]:
print(north_york_venues.shape)
north_york_venues

(240, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
1,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
2,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run
4,Hillcrest Village,43.803762,-79.363452,A.Y. Jackson Secondary School Track,43.805068,-79.366677,Athletics & Sports
5,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,43.777897,-79.344727,Shopping Mall
6,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,43.778207,-79.343483,Toy / Game Store
7,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Purdys Chocolatier,43.778099,-79.343853,Candy Store
8,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Apple Fairview,43.777952,-79.343582,Electronics Store
9,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,43.777082,-79.344557,Bakery


Let's check how many venues were returned for each neighborhood

In [63]:
north_york_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,4,4,4,4,4,4
"Emery, Humberlea",2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues

In [64]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 111 uniques categories.


<a id='item3'></a>

## 3. Analyze Each Neighborhood

In [44]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighbourhood'] = north_york_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()
# north_york_onehot

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [45]:
north_york_onehot.shape

(240, 112)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [46]:
north_york_grouped = north_york_onehot.groupby('Neighbourhood').mean().reset_index()
north_york_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tailor Shop,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.00,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.052632,0.052632,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.052632,0.052632,0.052632,0.000000,0.000000,0.0,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.0,0.038462,0.000000,0.000000,0.000000,0.000000,0.076923,0.038462,0.00,0.038462,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.076923,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0

#### Let's confirm the new size

In [47]:
north_york_grouped.shape

(24, 112)

#### Let's print each neighborhood along with the top 5 most common venues

In [48]:
num_top_venues = 5

for hood in north_york_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                venue  freq
0         Coffee Shop  0.11
1  Frozen Yogurt Shop  0.05
2            Pharmacy  0.05
3         Pizza Place  0.05
4                Park  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Fast Food Restaurant  0.08
1           Coffee Shop  0.08
2    Italian Restaurant  0.08
3              Pharmacy  0.04
4                  Café  0.04


----CFB Toronto, Downsview East----
      venue  freq
0   Airport  0.33
1      Park  0.33
2  Bus Stop  0.33
3     Plaza  0.00
4  Pharmacy  0.00


----Don Mills North----
                  venue  freq
0                  Pool  0.17
1  Gym / Fitness Center  0.17
2  Caribbean Restaurant  0.17
3                  Café  0.17
4   Japanese Restaurant  0.17


#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = north_york_grouped['Neighbourhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pet Store,Shopping Mall,Diner,Ice Cream Shop,Deli / Bodega,Park,Frozen Yogurt Shop,Pharmacy,Pizza Place
1,Bayview Village,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
2,"Bedford Park, Lawrence Manor East",Coffee Shop,Fast Food Restaurant,Italian Restaurant,Sandwich Place,Liquor Store,Comfort Food Restaurant,Café,Butcher,Pharmacy,Pizza Place
3,"CFB Toronto, Downsview East",Airport,Park,Bus Stop,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping
5,Downsview Central,Food Truck,Business Service,Baseball Field,Korean Restaurant,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
6,Downsview Northwest,Grocery Store,Athletics & Sports,Liquor Store,Discount Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
7,Downsview West,Grocery Store,Bank,Shopping Mall,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,"Emery, Humberlea",Furniture / Home Store,Baseball Field,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
9,"Fairview, Henry Farm, Oriole",Clothing Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Restaurant,Bus Station,Food Court,Dessert Shop,Liquor Store


<a id='item4'></a>

## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [51]:
# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 4, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [52]:
north_york_merged = north_york_df

# add clustering labels
north_york_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

north_york_merged # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,1,Dog Run,Athletics & Sports,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Food Truck,Coffee Shop,General Entertainment,Comfort Food Restaurant
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Restaurant,Bus Station,Food Court,Dessert Shop,Liquor Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,1,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,1,Gym,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,M2N,North York,Willowdale South,43.770120,-79.408493,1,Ramen Restaurant,Café,Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Lounge,Plaza,Pharmacy,Pet Store
6,M2P,North York,York Mills West,43.752758,-79.400049,1,Park,Bank,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
7,M2R,North York,Willowdale West,43.782736,-79.442259,1,Pizza Place,Pharmacy,Coffee Shop,Butcher,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
8,M3A,North York,Parkwoods,43.753259,-79.329656,4,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop
9,M3B,North York,Don Mills North,43.745906,-79.352188,1,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping


Finally, let's visualize the resulting clusters

In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighbourhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

## 5. Examine Clusters

Examining each cluster and determining the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster.

#### Cluster 1

In [54]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 0, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,North York,0,Airport,Park,Bus Stop,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
22,North York,0,Pizza Place,Empanada Restaurant,Discount Store,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega


#### Cluster 2

In [55]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Dog Run,Athletics & Sports,Pool,Mediterranean Restaurant,Golf Course,Fried Chicken Joint,Food Truck,Coffee Shop,General Entertainment,Comfort Food Restaurant
1,North York,1,Clothing Store,Fast Food Restaurant,Coffee Shop,Cosmetics Shop,Bakery,Restaurant,Bus Station,Food Court,Dessert Shop,Liquor Store
2,North York,1,Chinese Restaurant,Bank,Café,Japanese Restaurant,Furniture / Home Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Gift Shop,Cosmetics Shop
3,North York,1,Cafeteria,Women's Store,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
4,North York,1,Gym,Women's Store,Dog Run,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
5,North York,1,Ramen Restaurant,Café,Restaurant,Pizza Place,Coffee Shop,Sandwich Place,Lounge,Plaza,Pharmacy,Pet Store
6,North York,1,Park,Bank,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store
7,North York,1,Pizza Place,Pharmacy,Coffee Shop,Butcher,Clothing Store,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega
9,North York,1,Caribbean Restaurant,Basketball Court,Pool,Café,Japanese Restaurant,Gym / Fitness Center,Frozen Yogurt Shop,Diner,Comfort Food Restaurant,Construction & Landscaping
10,North York,1,Coffee Shop,Gym,Asian Restaurant,Beer Store,Sporting Goods Shop,Clothing Store,Chinese Restaurant,General Entertainment,Dim Sum Restaurant,Japanese Restaurant


#### Cluster 3

In [56]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2,Furniture / Home Store,Clothing Store,Event Space,Coffee Shop,Boutique,Accessories Store,Sporting Goods Shop,Vietnamese Restaurant,Diner,Comfort Food Restaurant


#### Cluster 4

In [57]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 3, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,North York,3,Grocery Store,Athletics & Sports,Liquor Store,Discount Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop,Deli / Bodega


#### Cluster 5

In [58]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 4, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,North York,4,Park,Food & Drink Shop,Fast Food Restaurant,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cupcake Shop


### Conclusion!

For cluster 1: Most common Venue: Airport

For cluster 2: Most common Venue: Dog run

For cluster 3: Most common Venue: Furniture and Home Store

For cluster 4: Most common Venue: Grocery Store

For cluster 5: Most common Venue: Food & Drink Shop

